In [14]:
from datasets import load_dataset
import pandas as pd
import ast


In [9]:
# ds = load_dataset("jhu-clsp/jfleg")
# ds.save_to_disk("JFLEG")
# ds["test"].to_csv("test.csv")
# ds["validation"].to_csv("validation.csv")

In [15]:
test_df = pd.read_csv("test.csv")
val_df  = pd.read_csv("validation.csv")

def parse_corrections(s):
    # Turn "['x', 'y']" into a Python list
    try:
        return ast.literal_eval(s)
    except Exception:
        return []

def comma_change_row(sentence, corrections):
    if not isinstance(sentence, str):
        return False
    if ',' not in sentence:
        return False
    orig_commas = sentence.count(',')
    for c in corrections:
        if not isinstance(c, str):
            continue
        # 1) Fewer commas in correction
        if c.count(',') < orig_commas:
            return True
        # 2) Punctuation changed (e.g., period or semicolon)
        if ('.' in c or ';' in c) and orig_commas > 0:
            return True
    return False

test_df["corr_list"] = test_df["corrections"].apply(parse_corrections)
val_df["corr_list"]  = val_df["corrections"].apply(parse_corrections)

test_df["comma_candidate"] = test_df.apply(
    lambda row: comma_change_row(row["sentence"], row["corr_list"]), axis=1
)
val_df["comma_candidate"] = val_df.apply(
    lambda row: comma_change_row(row["sentence"], row["corr_list"]), axis=1
)

test_comma = test_df[test_df["comma_candidate"]]
val_comma  = val_df[val_df["comma_candidate"]]